In [91]:
import os
import pandas
import nibabel as ni
import numpy as np
from glob import glob

In [15]:
sdir = '/Users/jakevogel/Downloads/normalized_microarray_donor9861/'
coords = pandas.read_csv(os.path.join(sdir,'SampleAnnot.csv'))
probe_info = pandas.read_csv(os.path.join(sdir,'Probes.csv'))
qc = pandas.read_csv(os.path.join(sdir,'PACall.csv'),header=None)

KeyboardInterrupt: 

In [16]:
data = pandas.read_csv(os.path.join(sdir,'MicroarrayExpression.csv'),header=None)

In [37]:
H1 = ni.load('/Users/jakevogel/Science/tau/tau_masks/shipping/tau_masks/vogel_figshare/winner_takes_all/TauRUS_WinnerTakesAll_Hippocampus_Off-Target.nii.gz'
            ).get_data()
H3 = ni.load('/Users/jakevogel/Science/tau/tau_masks/shipping/tau_masks/vogel_figshare/winner_takes_all/TauRUS_WinnerTakesAll_Hippocampus_Early_Tau.nii.gz'
            ).get_data()

In [35]:
ac = [(round(coords.mni_x[i]),
      round(coords.mni_y[i]),
      round(coords.mni_z[i])) for i in range(len(coords))]

In [42]:
ac[0][0],ac[0][1],ac[0][2]

(6.0, -28.0, 50.0)

In [40]:
H2[ac[0][0],ac[0][1],ac[0][2]]

//anaconda/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:1: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  if __name__ == '__main__':


0.0

In [41]:
[all_coords[x] for x in range(len(all_coords)) if H2[all_coords[x]] == 1]

//anaconda/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:1: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  if __name__ == '__main__':


[]

MNI SPACE

In [67]:
def convert_coords(coord, to_what = 'mni'):
    origin = [90, 126, 72]
    vs = 1
    x,y,z = coord[0],coord[1],coord[2]
    
    if to_what == 'mni':
        x = (origin[0]-x)*vs
        y = (y-origin[1])*vs
        z = (z-origin[2])*vs
        
    elif to_what == 'xyz':
        x=origin[0]- x/vs;
        y=y/vs + origin[1];
        z=z/vs + origin[2];
        
    else:
        raise IOError('please set to_what to either mni or xyz')
    
    return x,y,z

In [57]:
coords.columns

Index(['structure_id', 'slab_num', 'well_id', 'slab_type', 'structure_acronym',
       'structure_name', 'polygon_id', 'mri_voxel_x', 'mri_voxel_y',
       'mri_voxel_z', 'mni_x', 'mni_y', 'mni_z'],
      dtype='object')

In [60]:
a,b,c = coords.loc[coords.index[0]][['mni_x','mni_y','mni_z']].values
a

5.9000000000000004

In [80]:
hits_idx_3 = {}
for i in coords.index:
    mni_cs = coords.loc[i][['mni_x','mni_y','mni_z']]
    x,y,z = convert_coords(mni_cs, to_what = 'xyz')
    if H3[x,y,z] == 1:
        hits_idx_3.update({i: (x,y,z)})

//anaconda/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:5: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [81]:
hits_idx_1 = {}
for i in coords.index:
    mni_cs = coords.loc[i][['mni_x','mni_y','mni_z']]
    x,y,z = convert_coords(mni_cs, to_what = 'xyz')
    if H1[x,y,z] == 1:
        hits_idx_1.update({i: (x,y,z)})

//anaconda/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:5: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [82]:
hits_idx_3.keys()

dict_keys([306, 532, 533, 582, 580, 554, 796, 44, 530, 573])

In [86]:
data.iloc[:,list(hits_idx_3.keys())].mean(axis=1)

0         2.643289
1         1.560705
2         1.676049
3         5.373549
4         7.523561
5         8.377252
6        10.950905
7         5.883881
8         7.992006
9         3.615337
10       12.174057
11        9.849032
12       11.117673
13        3.000522
14        3.789344
15        1.775950
16        9.644368
17        5.881371
18        4.501402
19       10.325309
20       10.384866
21        5.459606
22        5.110865
23        6.946877
24        9.497831
25        8.875330
26        9.408522
27        6.886190
28        6.759526
29        9.631543
           ...    
58662     1.820298
58663     2.599219
58664     1.656004
58665     1.994733
58666     7.298187
58667     1.756525
58668    10.188729
58669     1.987665
58670     2.193815
58671    10.340270
58672     2.661737
58673     1.843563
58674     3.194600
58675     9.699841
58676     1.587918
58677     1.674414
58678     7.990807
58679     1.712902
58680    10.215604
58681     1.607155
58682     6.262849
58683     4.

In [89]:
def ABA_average_expression_from_atlas(ABA_dir, atlas, binary=False, save=False):
    
    ABA_subs = sorted(glob(os.path.join(ABA_dir,'normalized_microarray_donor*')))
    final_out = {}
    for sdir in ABA_subs:
        sid = os.path.join(sdir)[-1].split('donor')[-1]
        print('working on subject',sid)
        print('loading data...')
        coords = pandas.read_csv(os.path.join(sdir,'SampleAnnot.csv'))
        if binary:
            data = pandas.read_csv(os.path.join(sdir,'PACall.csv'),header=None)
        else:
            data = pandas.read_csv(os.path.join(sdir,'MicroarrayExpression.csv'),header=None)
        
        print('preparing image...')
        atl = ni.load(atlas).get_data()
        output = pandas.DataFrame(index = data.index)
        for lab in np.unique(atl)[1:]:
            print('searching for probes in label',lab)
            hits_idx = {}
            for i in coords.index:
                mni_cs = coords.loc[i][['mni_x','mni_y','mni_z']]
                x,y,z = convert_coords(mni_cs, to_what = 'xyz')
                if atl[x,y,z] == lab:
                    hits_idx.update({i: (x,y,z)})
            print('found %s hits'%(len(hits_idx.keys())))
            output.loc[:,'lab_%s'%lab] = data.iloc[:,list(hits_idx.keys())].mean(axis=1)
        print('finishing up subject',sid)
        final_out.update({sid: output})
        if save:
            output.to_csv(os.path.join(ABA_dir,'%s_avg_expression_data.csv'%sid))
        
    return final_out

In [92]:
ABA_dir = '/Users/jakevogel/Downloads/'
atlas = '/Users/jakevogel/Science/tau/tau_masks/1mm_lund/hippocampus_labels_LUND_scif55.nii.gz'
t1 = ABA_average_expression_from_atlas(ABA_dir, atlas)

working on subject 1
loading data...
preparing image...
searching for probes in label 1.0


//anaconda/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:24: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


found 39 hits
searching for probes in label 3.0
found 10 hits
finishing up subject 1


In [94]:
t1['1']

,lab_1.0,lab_3.0
0,2.968873,2.643289
1,1.725170,1.560705
2,1.750512,1.676049
3,4.958166,5.373549
4,7.331524,7.523561
5,8.040014,8.377252
6,10.630404,10.950905
7,4.690699,5.883881
8,8.999065,7.992006
9,2.885057,3.615337
